In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [0]:
class CartPoleNet(nn.Module): 
  def __init__(self):
    super(CartPoleNet, self).__init__()
    self.conv1 = nn.Conv3d(3, 6, 3, padding=1) 
    self.conv2 = nn.Conv3d(6, 16, 3, padding=1)
    self.conv3 = nn.Conv3d(16, 32, 3, padding=1)
    self.conv4 = nn.Conv3d(32, 64, 3, padding=1)
    # fully connected layers, make sure to add up images here
    self.fc1 = nn.Linear(8 * 8 * 64, 120)
    self.fc2 = nn.Linear(120, 100)
    self.fc3 = nn.Linear(100, 50)
    self.fc4 = nn.Linear(50, 4)

  def forward(self, x):
    x = F.max_pool3d(F.relu(self.conv1(x)), (1, 2, 2)) # 64
    x = F.max_pool3d(F.relu(self.conv2(x)), (1, 2, 2)) # 32
    x = F.max_pool3d(F.relu(self.conv3(x)), (1, 2, 2)) # 16
    x = F.max_pool3d(F.relu(self.conv4(x)), (1, 2, 2)) # 8
    # sum up along batch axis before moving onto fully connected layers
    x = torch.sum(x, dim=2)
    x = x.view(-1, self.num_flat_features(x))
    # fully connected layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features


In [0]:
input = torch.randn(5, 3, 5, 128, 128)
net = CartPoleNet()

In [0]:
out = net(input)

In [17]:
print(out)

tensor([[ 0.1188, -0.0678, -0.1156, -0.0818],
        [ 0.1178, -0.0672, -0.1155, -0.0811],
        [ 0.1180, -0.0673, -0.1161, -0.0804],
        [ 0.1191, -0.0654, -0.1150, -0.0808],
        [ 0.1189, -0.0670, -0.1162, -0.0811]], grad_fn=<AddmmBackward>)
